In [5]:
#%pip install tensorflow
import tensorflow as tf
import pandas as pd
## Load in data

tickets = pd.read_csv("master_dataset.csv")

<bound method NDFrame.head of        Unnamed: 0        tag  \
0               0  ZOOKEEPER   
1               1  ZOOKEEPER   
2               2  ZOOKEEPER   
3               3  ZOOKEEPER   
4               4  ZOOKEEPER   
...           ...        ...   
87981       87981   CKEDITOR   
87982       87982   CKEDITOR   
87983       87983   CKEDITOR   
87984       87984   CKEDITOR   
87985       87985   CKEDITOR   

                                                    desc  effort(s)  
0              Upgrade `cyclonedx-maven-plugin` to 2.7.6       4800  
1            Improve SuppressWarnings in KeeperException       1800  
2      Thus we don't need to do some new FileSnap(nul...       1800  
3      OpCode.check is treated as a quorum write oper...       1200  
4      Failure: We need to run apt update before inst...       2400  
...                                                  ...        ...  
87981  It happens more often on Chrome than on Firefo...       7200  
87982  Steps to reproduce

In [7]:
import numpy as np

def oneHotEncoder(vocab, input):
    ## Make an array size of the vocab
    encoderMatrix = np.zeros(len(vocab))

    ## Iterate the input strings and encode their position in the array
    for word in input:
        index = vocab[vocab['word']==word].index.values
        encoderMatrix[index] += 1
    
    return encoderMatrix

## Method for getting input
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import stopwords
import nltk
nltk.download("punkt")
nltk.download('wordnet')
nltk.download("stopwords")

def parse_text(text):

    ## Tokenize string into words (and punctuation)
    word_array = word_tokenize(text)
    word_array = [word.lower() for word in word_array if word.isalpha()]

    ## Filter out stop words
    stop_words = set(stopwords.words("english"))
    filtered_words = [word for word in word_array if word.casefold() not in stop_words]

    ## Turn words into lemmatized words
    lemmatizer = WordNetLemmatizer()
    lemitized_words = [lemmatizer.lemmatize(word) for word in filtered_words]

    ## Apply Stemming (Find the roots of similar words)
    stemmer = PorterStemmer()
    stemmed_words = [stemmer.stem(word) for word in lemitized_words]

    return stemmed_words

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Josh\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Josh\AppData\Roaming\nltk_data...
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Josh\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [28]:

vocab = pd.read_csv("reduced_vocabulary.csv")
#vocab = vocab['word'].tolist()
encodings = []
efforts = []

for i in range(len(tickets)):
    parsed = parse_text(tickets.loc[i, ['desc']].values[0])
    encoding = oneHotEncoder(vocab, parsed)
    encodings.append(encoding)
    efforts.append(tickets.loc[i, ['effort(s)']].values[0])


print(encodings)
print(efforts)

[array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 2., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 1., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 1., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 1., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 

In [29]:
## Store to save on computation time later

data = {'Encodings': encodings, 'effort': efforts}
df = pd.DataFrame(data)
df.to_csv("encodings_and_effort.csv")

In [52]:

## Gather training data
from sklearn.model_selection import train_test_split
labels = pd.read_csv("labelled_dataset.csv")

labels = labels['Grade'].tolist()
labels = [label - 1 for label in labels]

data = {'encoding': encodings, 'effort': labels}
df = pd.DataFrame(data)

train, test = train_test_split(df, test_size=0.2)

x_train = tf.convert_to_tensor(train['encoding'].to_list())
y_train = tf.convert_to_tensor(train['effort'])

x_test = tf.convert_to_tensor(test['encoding'].to_list())
y_test = tf.convert_to_tensor(test['effort'])

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

print(x_test)

print(len(x_train[0]))



tf.Tensor(
[[0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], shape=(17598, 2742), dtype=float64)
2742


In [64]:
## SETUP NEURAL NETWORK

from tensorflow import keras

model = keras.Sequential([
    keras.layers.Flatten(input_shape=(2742,1)),
    keras.layers.Dense(500, activation="relu"),
    keras.layers.Dense(500, activation="relu"),
    keras.layers.Dense(500, activation="relu"),
    keras.layers.Dense(3, activation="softmax")
])

model.compile(optimizer="SGD", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

model.fit(x_train, y_train, epochs=100)

test_loss, test_acc = model.evaluate(x_test, y_test)

print("Tested Acc: ", test_acc)

Epoch 1/15
2200/2200 [==============================] - 9s 4ms/step - loss: 1.0051 - accuracy: 0.5449
Epoch 2/15
2200/2200 [==============================] - 9s 4ms/step - loss: 1.0002 - accuracy: 0.5450
Epoch 3/15
2200/2200 [==============================] - 9s 4ms/step - loss: 0.9972 - accuracy: 0.5451
Epoch 4/15
2200/2200 [==============================] - 9s 4ms/step - loss: 0.9926 - accuracy: 0.5467
Epoch 5/15
2200/2200 [==============================] - 9s 4ms/step - loss: 0.9852 - accuracy: 0.5534
Epoch 6/15
2200/2200 [==============================] - 9s 4ms/step - loss: 0.9747 - accuracy: 0.5613
Epoch 7/15
2200/2200 [==============================] - 9s 4ms/step - loss: 0.9629 - accuracy: 0.5686
Epoch 8/15
2200/2200 [==============================] - 9s 4ms/step - loss: 0.9524 - accuracy: 0.5742
Epoch 9/15
2200/2200 [==============================] - 9s 4ms/step - loss: 0.9452 - accuracy: 0.5781
Epoch 10/15
2200/2200 [==============================] - 8s 4ms/step - loss: 0.940